In [1]:
"""
Adapted from keras example cifar10_cnn.py
Train ResNet-18 on the CIFAR10 small images dataset.

GPU run command with Theano backend (with TensorFlow, the GPU is automatically used):
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python cifar10.py
"""
from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping

import numpy as np
import resnet


lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
csv_logger = CSVLogger('resnet18_cifar10.csv')

batch_size = 32
nb_classes = 10
#nb_epoch = 200
nb_epoch = 50
data_augmentation = True

# input image dimensions
img_rows, img_cols = 32, 32
# The CIFAR10 images are RGB.
img_channels = 3 

# The data, shuffled and split between train and test sets:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Convert class vectors to binary class matrices.
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# subtract mean and normalize
mean_image = np.mean(X_train, axis=0)
X_train -= mean_image
X_test -= mean_image
X_train /= 128.
X_test /= 128.

model = resnet.ResnetBuilder.build_resnet_18((img_channels, img_rows, img_cols), nb_classes)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(X_test, Y_test),
              shuffle=True,
              callbacks=[lr_reducer, early_stopper, csv_logger])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(X_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        validation_data=(X_test, Y_test),
                        epochs=nb_epoch, verbose=1, max_q_size=100,
                        callbacks=[lr_reducer, early_stopper, csv_logger])

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Using real-time data augmentation.
Instructions for updating:
Use tf.cast instead.


/home/ktran/miniconda3/envs/art/lib/python2.7/site-packages/ipykernel_launcher.py:87: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., verbose=1, validation_data=(array([[[..., steps_per_epoch=1562, epochs=50, callbacks=[<keras.ca..., max_queue_size=100)`


Epoch 1/50
1562/1562 [==============================] - 36s 23ms/step - loss: 1.7972 - acc: 0.4874 - val_loss: 1.4438 - val_acc: 0.5928
Epoch 2/50
1562/1562 [==============================] - 32s 20ms/step - loss: 1.3851 - acc: 0.6084 - val_loss: 1.5200 - val_acc: 0.5903
Epoch 3/50
1562/1562 [==============================] - 32s 21ms/step - loss: 1.2325 - acc: 0.6577 - val_loss: 1.1047 - val_acc: 0.6985
Epoch 4/50
1562/1562 [==============================] - 32s 21ms/step - loss: 1.1375 - acc: 0.6867 - val_loss: 1.1122 - val_acc: 0.6890
Epoch 5/50
1562/1562 [==============================] - 32s 21ms/step - loss: 1.0732 - acc: 0.7083 - val_loss: 1.1592 - val_acc: 0.6827
Epoch 6/50
1562/1562 [==============================] - 32s 21ms/step - loss: 1.0270 - acc: 0.7248 - val_loss: 1.0623 - val_acc: 0.7086
Epoch 7/50
1562/1562 [==============================] - 32s 21ms/step - loss: 0.9957 - acc: 0.7355 - val_loss: 1.1708 - val_acc: 0.6880
Epoch 8/50
1562/1562 [==========================

In [15]:
np.argmax(model.predict(X_test)[0])

0

In [16]:
y_test[0][0]

3